# Training using Pandas dataframes
In this notebook we'll take a look at how to use pandas dataframes to train a neural network in CNTK.
We're reusing the code from chapter 1 where we trained a iris classification model.

## Building the model
The model we use here is a basic classification model with a single hidden layer and an output layer that understands three possible species of flowers. We use a softmax activation function on the output layer and a sigmoid function on the hidden layer.

In [85]:
from cntk import default_options, input_variable
from cntk.layers import Dense, Sequential
from cntk.ops import log_softmax, relu, sigmoid

model = Sequential([
    Dense(4, activation=sigmoid),
    Dense(3, activation=log_softmax)
])

features = input_variable(4)
labels = input_variable(3)

z = model(features)

## Loading and preprocessing the training data
We load up the iris dataset and preprocess it so that we end up with a set of floating point numbers as required by the model.
Please note that we didn't split the dataset, this sample is just to demonstrate that you can use pandas dataframes. It is important to know that you can use `train_test_split` from the `scikit-learn` library to perform the necessary splitting.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer

df_source = pd.read_csv('iris.csv', 
    names=['sepal_length', 'sepal_width','petal_length','petal_width', 'species'], 
    index_col=False)

label_encoder = LabelBinarizer()

X = df_source.iloc[:, :4].values
y = df_source.iloc[:, -1:].values

y = label_encoder.fit_transform(y)

X = X.astype(np.float32)
y = y.astype(np.float32)

## Training the model
In the previous step we've converted our data from pandas to numpy arrays. This makes it fairly straightforward to train our model. We need a loss and learner to optimize the model. We've included a progress printer so we can see what the training process looks like.

In [87]:
from cntk.losses import cross_entropy_with_softmax
from cntk.learners import sgd 
from cntk.logging import ProgressPrinter

progress_writer = ProgressPrinter(0)
loss = cross_entropy_with_softmax(z, labels)
learner = sgd(z.parameters, 0.1)

train_summary = loss.train((X,y), parameter_learners=[learner], callbacks=[progress_writer], minibatch_size=16, max_epochs=5)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.1
     1.84       1.84          0          0            16
     1.51       1.34          0          0            48
      1.5       1.49          0          0           112
     1.43       1.43          0          0            16
     1.25       1.16          0          0            48
     1.31       1.35          0          0           112
     1.38       1.38          0          0            16
     1.22       1.14          0          0            48
     1.26       1.29          0          0           112
     1.38       1.38          0          0            16
     1.22       1.14          0          0            48
     1.25       1.27          0          0           112
     1.38       1.38          0          0            16
     1.22       1.14          0          0            48
 